# 待解問題
- 時間
- 內文格式不同的狀況

In [116]:
import re
import json
import pandas as pd
import time
from collections import defaultdict
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

In [11]:
# 登入
# username = browser.find_element_by_name('email')
# password = browser.find_element_by_name('pass')
# username.send_keys('kobe821125@gmail.com')
# password.send_keys('calvin19931125')

# log_in_button = browser.find_element_by_name('login')
# log_in_button.click()

In [157]:
browser = webdriver.Chrome()
browser.get('https://www.facebook.com/groups/1189431674567215/')
html = browser.page_source

In [158]:
# 滾動卷軸
browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

In [175]:
# 時間
date = browser.find_elements_by_css_selector(".b6zbclly.myohyog2.l9j0dhe7.aenfhxwr.l94mrbxd.ihxqhq3m.nc684nl6.t5a262vz.sdhka5h4")
timestamp = []
for i in date:
    if i.text is not '':
        print(i.text)
        timestamp.append(i.text)

2020年12月19日
2020年12月19日
7 小時
7 小時


In [176]:
timestamp[-1]

'7 小時'

In [181]:
if re.search(r'小時', timestamp[-1]):
    print(timestamp[-1])

7 小時


In [161]:
# 點擊查看更多
read_button = browser.find_elements_by_css_selector("div[class='oajrlxb2 g5ia77u1 qu0x051f esr5mh6w e9989ue4 r7d6kgcz rq0escxv nhd2j8a9 nc684nl6 p7hjln8o kvgmc6g5 cxmmr5t8 oygrvhab hcukyx3x jb3vyjys rz4wbd8a qt6c0cv9 a8nywdso i1ao9s8h esuyzwwr f1sip0of lzcic4wl oo9gr5id gpro0wi8 lrazzd5p'][role='button']")
for read in read_button:
    time.sleep(1)
    read.click()

In [163]:
title = browser.find_elements_by_css_selector("div[class='kvgmc6g5 cxmmr5t8 oygrvhab hcukyx3x c1et5uql ii04i59q']")
value = browser.find_elements_by_css_selector(".o9v6fnle.cxmmr5t8.oygrvhab.hcukyx3x.c1et5uql.ii04i59q")

In [164]:
# test
for i, t in enumerate(title):
    if t.text is not '':
        content = t.text
        code = re.search(r'\d{4}', content)#.group()
        
        if code:
            code = code.group()
            name = re.split(r'\(', content)[0].strip()
            print(name, code)
            
        nums = i

In [165]:
nums

3

In [166]:
# 公司代號
def company_name(title):
    output = defaultdict(list)
    for t in title:
        if t.text is not '':
            content = t.text
            code = re.search(r'\d{4}', content)
            
            if code:
                code = code.group()
                name = re.split(r'\(', content)[0].strip()          
                output[name].append(code)
                
    return output

stock_info = company_name(title)
stock_info

defaultdict(list, {})

In [145]:
def company_analysis_idx(value):
    company_index = defaultdict(list)
    for index ,v in enumerate(value):
        if v.text is not '':
            info = v.text
            company = info.split('的')[0].strip()     
            if company in stock_info.keys():
                company_index[company].append(index)
            #print(index, company)
    return company_index

stock_idx = company_analysis_idx(value)

In [146]:
stock_idx

defaultdict(list,
            {'國泰金': [39],
             '富邦金': [42],
             '統一超': [45],
             '卓越': [51],
             '東元': [53],
             '由田': [54],
             '富邦媒': [55],
             '八貫': [57],
             '日月光': [58],
             '中保科': [59]})

In [147]:
def company_analysis_content(stock_idx, value):
    
    for stock in stock_idx.keys():
        indexs = value[stock_idx[stock][0]].text.split('\n')
        for index in indexs:
            index = index.split('的')[1]
            stock_info[stock].append(index)
            
        stock_info[stock].append(value[stock_idx[stock][0]+1].text)

In [148]:
company_analysis_content(stock_idx, value)
stock_info

IndexError: list index out of range

In [ ]:
o9v6fnle cxmmr5t8 oygrvhab hcukyx3x c1et5uql ii04i59q
o9v6fnle cxmmr5t8 oygrvhab hcukyx3x c1et5uql ii04i59q
kvgmc6g5 cxmmr5t8 oygrvhab hcukyx3x c1et5uql ii04i59q
kvgmc6g5 cxmmr5t8 oygrvhab hcukyx3x c1et5uql ii04i59q
kvgmc6g5 cxmmr5t8 oygrvhab hcukyx3x c1et5uql ii04i59q
o9v6fnle cxmmr5t8 oygrvhab hcukyx3x c1et5uql ii04i59q
b6zbclly myohyog2 l9j0dhe7 aenfhxwr l94mrbxd ihxqhq3m nc684nl6 t5a262vz sdhka5h4